### Download the dataset about Mnist

#### 导入相关包

In [18]:
import torch
from torch import nn
from torch.autograd.variable import Variable

import torchvision
import torchvision.transforms as transforms

In [19]:
# Preprocess
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]
)

In [20]:
# Training data
train_set = torchvision.datasets.MNIST(
    root='.', train=True, download=True, transform=transform
)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=32, shuffle=True
)

判别器部分。 判别器网络是对图像真实与否进行分类。  
输入：28×28像素 -> 一个长度为784的向量  
输出：一个单独的值<表示图像是否是实际的MNIST数字>  

In [21]:
# Our Discriminator classes
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.model(x.view(x.size(0), 784))
        out = out.view(out.size(0), -1)
        return out.cuda()

discriminator = Discriminator()

生成器部分。 生成器网络负责创建实际的图像。  
输入：一个长度为100的向量<纯随机噪声>  
输出：一个长度为784的向量 -> 28×28像素  

In [22]:
# Our Generator class
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, x):
        x = x.view(x.size(0), 100)
        out = self.model(x)
        return out

generator = Generator()

把模型移动到GPU上

In [23]:
# If we have a GPU with CUDA, use it
if torch.cuda.is_available():
    print("Using CUDA")
    discriminator.cuda()
    generator.cuda()

# Setup loss function and optimizers
lr = 0.01
num_epochs = 40
num_batches = len(train_loader)

criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

Using CUDA


开始训练循环。
训练GAN的关键是我们需要在一个循环中更新生成器和判别器。

In [24]:
# Convenience function for training our Discriminator
def train_discriminator(discriminator, real_images, real_labels, fake_images, fake_labels):
    discriminator.zero_grad()

    # Get the predictions, loss, and score of the real images
    predictions = discriminator(real_images)
    real_loss = criterion(predictions, real_labels)
    real_score = predictions

    # Get the predictions, loss, and score of the fake images
    predictions = discriminator(fake_images)
    fake_loss = criterion(predictions, fake_labels)
    fake_score = predictions

    # Calculate the total loss, update the weights, and update the optimizer
    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss, real_score, fake_score

In [25]:
# Convenience function for training our Generator
def train_generator(generator, discriminator_outputs, real_labels):
    generator.zero_grad()

    # Calculate the total loss, update the weights, and update the optimizer
    g_loss = criterion(discriminator_outputs, real_labels)
    g_loss.backward()
    g_optimizer.step()
    return g_loss

In [26]:
for epoch in range(num_epochs):
    for n, (images, _) in enumerate(train_loader):
        # (1) Prepare the real data for the Discriminator
        real_images = Variable(images).cuda()
        real_labels = Variable(torch.ones(images.size(0))).reshape([32, 1]).cuda()

        # (2) Prepare the random noise data for the Generator
        noise = Variable(torch.randn(images.size(0), 100)).cuda()

        # (3) Prepare the fake data for the Discriminator
        fake_images = generator(noise)
        fake_labels = Variable(torch.zeros(images.size(0))).reshape([32, 1]).cuda()

        # (4) Train the discriminator on real and fake data
        d_loss, real_score, fake_score = train_discriminator(
            discriminator,
            real_images, real_labels,
            fake_images, fake_labels
        )

        # (5a) Generate some new fake images from the Generator.
        # (5b) Get the label predictions of the Discriminator on that fake data.
        noise = Variable(torch.randn(images.size(0), 100)).cuda()
        fake_images = generator(noise)

        outputs = discriminator(fake_images)

        # (6) Train the generator
        g_loss = train_generator(generator, outputs, real_labels)

# Save the model
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

In [30]:
#推理
import torch
from torch.autograd.variable import Variable
from torchvision.utils import save_image

# Load the model

generator = Generator()
generator.load_state_dict(torch.load('generator.pth'))
generator.eval()
generator.cuda()

# Generate some random noise
noise = Variable(torch.randn(32, 100)).cuda()

# Generate the images
fake_images = generator(noise)

# Save the images
save_image(fake_images.data.view(32, 1, 28, 28), 'fake_images.png')


